In [1]:
from Bio.PDB import PDBList
import json

Справка по arpeggio (нажать View as a scrollable element)

In [2]:
%%bash
conda run -n arpeggio-env pdbe-arpeggio -h

usage: pdbe-arpeggio [-h] [-s SELECTION [SELECTION ...] | -sf SELECTION_FILE]
                     [-wh] [-mh] [-ms MINIMISATION_STEPS]
                     [-mf {MMFF94,UFF,Ghemical}]
                     [-mm {DistanceGeometry,SteepestDescent,ConjugateGradients}]
                     [-co VDW_COMP] [-i INTERACTING] [-ph PH] [-sa] [-a]
                     [-o OUTPUT] [-m]
                     filename

        ############
        # ARPEGGIO #
        ############

        A library for calculating interatomic contacts in proteins

        Dependencies:
        - Python >= 3.6
        - Numpy
        - BioPython >= v1.80
        - Open Babel >= 3.0.0
        - gemmi >= 0.5.8

        

positional arguments:
  filename              Path to the file to be analysed.

optional arguments:
  -h, --help            show this help message and exit
  -s SELECTION [SELECTION ...], --selection SELECTION [SELECTION ...]
                        Select the "ligand" for interactions, using selection

Этот способ не работает! 
arpeggio потом выдает ошибку 
"ValueError: The cif file does not contain _atom_site record"

С помощью open babel конвертирую окружение вокруг остатка в формат .cif

In [ ]:
# %%bash
# mkdir -p ./cutCIFs/
# conda run -n arpeggio-env obabel './PDBs/1FC2.pdb' -O './cutCIFs/1FC2_F149A.cif' # ?-h?

Буду загружать mmcif с Protein Data Bank...

In [4]:
def download_cif(pdb_id):
    pdb_list = PDBList()
    pdb_list.retrieve_pdb_file(pdb_id, pdir="./CIFs", file_format="mmCif", overwrite=True)
    print(f"Файл {pdb_id}.cif успешно скачан")

In [5]:
download_cif('1FC2')

Файл 1FC2.cif успешно скачан


В статье mCSM-AB2 написано, что брали interatomic contacts, но не указано какие атомы. Полагаю, что между атомами мутируемой АК и всем остальным вокруг (включая цепь, на которой АК сидит). То есть нужно эту аминокислоту указать как 'лиганд' c опцией -s

In [6]:
pdb_id = '1FC2'
chain = 'C'
aa_num = 149

In [7]:
!conda run -n arpeggio-env pdbe-arpeggio -s /{chain}/{aa_num}/ -o './arp_contacts_json/' '/Users/chekalin/Desktop/AffinityTool/feature_eng/cutCIFs/{pdb_id}.cif'

INFO//01:15:09.529//Program begin.
INFO//01:15:09.529//Selection perceived: ['/C/149/']
Traceback (most recent call last):
  File "/Users/chekalin/miniforge3/envs/arpeggio-env/bin/pdbe-arpeggio", line 8, in <module>
    sys.exit(main())
  File "/Users/chekalin/miniforge3/envs/arpeggio-env/lib/python3.9/site-packages/arpeggio/scripts/process_protein_cli.py", line 150, in main
    run_arpeggio(args)
  File "/Users/chekalin/miniforge3/envs/arpeggio-env/lib/python3.9/site-packages/arpeggio/scripts/process_protein_cli.py", line 159, in run_arpeggio
    i_complex = InteractionComplex(
  File "/Users/chekalin/miniforge3/envs/arpeggio-env/lib/python3.9/site-packages/arpeggio/core/interactions.py", line 53, in __init__
    self.biopython_str = self._read_in_biopython(filename)
  File "/Users/chekalin/miniforge3/envs/arpeggio-env/lib/python3.9/site-packages/arpeggio/core/interactions.py", line 2032, in _read_in_biopython
    else protein_reader.read_mmcif_to_biopython(path))
  File "/Users/cheka

Заметил, что не все контакты в полученном .json файле содержат в участниках интересуемую АК (что странно)

Контакты, которые включают интересуемую АК имеют значение "INTER" в поле "interacting_entities",
другие — "INTRA_NON_SELECTION"

Отфильтрую все такие контакты

In [8]:
# Загрузка данных из JSON файла
with open(f'./arp_contacts_json/{pdb_id}.json', 'r') as file:
    contacts = json.load(file)

# Фильтрация списка, удаление словарей с "interacting_entities": "INTRA_NON_SELECTION"
filt_contacts = list(filter(lambda x: x.get('interacting_entities') != 'INTRA_NON_SELECTION', contacts))

# Сохранение отфильтрованных данных обратно в файл
with open(f'./arp_contacts_json/{pdb_id}.json', 'w') as file:
    json.dump(filt_contacts, file, indent=4)

print("Фильтрация завершена")

Фильтрация завершена


In [9]:
filt_contacts

[{'bgn': {'auth_asym_id': 'C',
   'auth_atom_id': 'CG2',
   'auth_seq_id': 135,
   'label_comp_id': 'ILE',
   'label_comp_type': 'P',
   'pdbx_PDB_ins_code': ' '},
  'contact': ['proximal'],
  'distance': 4.43,
  'end': {'auth_asym_id': 'C',
   'auth_atom_id': 'C',
   'auth_seq_id': 149,
   'label_comp_id': 'PHE',
   'label_comp_type': 'P',
   'pdbx_PDB_ins_code': ' '},
  'interacting_entities': 'INTER',
  'type': 'atom-atom'},
 {'bgn': {'auth_asym_id': 'C',
   'auth_atom_id': 'CD2',
   'auth_seq_id': 153,
   'label_comp_id': 'LEU',
   'label_comp_type': 'P',
   'pdbx_PDB_ins_code': ' '},
  'contact': ['proximal'],
  'distance': 4.81,
  'end': {'auth_asym_id': 'C',
   'auth_atom_id': 'C',
   'auth_seq_id': 149,
   'label_comp_id': 'PHE',
   'label_comp_type': 'P',
   'pdbx_PDB_ins_code': ' '},
  'interacting_entities': 'INTER',
  'type': 'atom-atom'},
 {'bgn': {'auth_asym_id': 'C',
   'auth_atom_id': 'C',
   'auth_seq_id': 149,
   'label_comp_id': 'PHE',
   'label_comp_type': 'P',
   '